In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import numpy as np
import pandas as pd
from alive_progress import alive_it
from sklearn.metrics import classification_report

In [2]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [3]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
data_path = "Sentiment-Analysis/"
test_data = pd.read_csv(data_path + "test.csv")
test_data = test_data[['textID', 'text', 'sentiment']]
test_data = test_data.dropna()

In [5]:
test_texts = list(test_data['text'].values)
test_sentiments = list(test_data['sentiment'].values)

In [6]:
labels = []
for test_text in alive_it(test_texts, force_tty=True, max_cols=120):
    text = preprocess(test_text)
    encoded_input = tokenizer(text,
                              return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    label = config.id2label[ranking[0]]
    score = scores[ranking[0]]
    
    labels.append({
        'label': label,
        'score': score
    })

|████████████████████████████████████████| 3534/3534 [100%] in 2:31.7 (23.30/s)                                         


In [7]:
test_pred = [label['label']
             for label in labels]

print(classification_report(test_pred,
                            test_sentiments))

              precision    recall  f1-score   support

    negative       0.81      0.69      0.74      1181
     neutral       0.51      0.77      0.61       944
    positive       0.88      0.69      0.78      1409

    accuracy                           0.71      3534
   macro avg       0.73      0.72      0.71      3534
weighted avg       0.76      0.71      0.72      3534

